In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder


In [2]:
train_data = pd.read_csv('./train.csv')
test_data = pd.read_csv('./test.csv')
real_data = pd.read_csv('./sample_submission.csv')

merged_data = pd.concat([train_data, test_data], axis=0).reset_index(drop=True)

In [3]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             2919 non-null   int64  
 1   MSSubClass     2919 non-null   int64  
 2   MSZoning       2915 non-null   object 
 3   LotFrontage    2433 non-null   float64
 4   LotArea        2919 non-null   int64  
 5   Street         2919 non-null   object 
 6   Alley          198 non-null    object 
 7   LotShape       2919 non-null   object 
 8   LandContour    2919 non-null   object 
 9   Utilities      2917 non-null   object 
 10  LotConfig      2919 non-null   object 
 11  LandSlope      2919 non-null   object 
 12  Neighborhood   2919 non-null   object 
 13  Condition1     2919 non-null   object 
 14  Condition2     2919 non-null   object 
 15  BldgType       2919 non-null   object 
 16  HouseStyle     2919 non-null   object 
 17  OverallQual    2919 non-null   int64  
 18  OverallC

In [4]:
def encode_categorical_columns(df):
    categorical_columns = df.select_dtypes(include=['object']).columns
    le = LabelEncoder()
    encoded_columns = []
    for column in categorical_columns:
        df[column] = le.fit_transform(df[column])
        encoded_columns.append(column)

    return df

In [5]:
merged_data = encode_categorical_columns(merged_data)

/Users/balkanskihaker/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/balkanskihaker/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/balkanskihaker/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/balkanskihaker/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd

In [6]:
merged_data = pd.get_dummies(merged_data, dtype=int)

In [7]:
is_missing = merged_data.isna().any().replace({False: None}).dropna().index.to_list()

In [8]:
corr_missing = merged_data.corr()
#corr_missing = corr_missing[is_missing]
corr_missing.replace({1.0: None}, inplace=True)

In [9]:

#Sale_price_corr = corr_missing['SalePrice'][corr_missing['SalePrice'].sort_values(ascending=False) > 0.35]
#important_for_sale = Sale_price_corr.index
#merged_data[important_for_sale].info()

In [10]:
for column in corr_missing.columns:

    if merged_data[column].isnull().sum() == 0:
        continue

    X_columns_all = corr_missing[column].dropna().abs().sort_values(ascending=False)[:5].index.to_list()
    X_columns = []

    for header in X_columns_all:
        if merged_data[header].isna().sum() == 0:
            X_columns.append(header)

    n_neighbors = len(X_columns)
    knn_regressor = KNeighborsRegressor(n_neighbors=n_neighbors)

    X = merged_data[X_columns]
    X = pd.concat([X, merged_data['Id']], axis=1)
    X.dropna(axis=0, inplace=True)

    y = pd.DataFrame(merged_data[column])
    y = pd.concat([y, merged_data['Id']], axis=1)
    y.dropna(axis=0, inplace=True)

    X = X[X['Id'].isin(y['Id'])]
    y = y[y['Id'].isin(X['Id'])]

    X_columns.append('Id')
    X_missing = merged_data[merged_data[column].isnull()][X_columns]

    columns = [column, 'Id']
    y_missing = merged_data[~merged_data[column].notna()][columns]

    knn_regressor.fit(X.drop('Id', axis=1), y[column])

    y_missing[column] = knn_regressor.predict(X=X_missing.drop('Id', axis=1))

    #merged_data.loc[merged_data['Id'].isin(y_missing), column] = y_missing[column].values
    merged_data.loc[merged_data['Id'].isin(y_missing['Id']), column] = y_missing[column][:len(y_missing['Id'])]

/Users/balkanskihaker/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/balkanskihaker/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/balkanskihaker/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/balkanskihaker/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed

In [11]:
y = train_data['SalePrice']
merged_data_indexes = merged_data['Id']
merged_data.drop('Id', axis=1, inplace=True)

In [12]:
#scaler = StandardScaler()
#scaled_data = scaler.fit_transform(merged_data)
#scaled = pd.DataFrame(scaled_data, columns=merged_data.columns)
#scaled['Id'] = merged_data_indexes

/Users/balkanskihaker/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/balkanskihaker/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/balkanskihaker/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/balkanskihaker/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed

In [13]:
train_data_edited = merged_data.iloc[:1460]
test_data_edited = merged_data.iloc[1460:]

In [17]:
model = RandomForestRegressor()

model.fit(X=train_data_edited, y=y)

prediction = model.predict(test_data_edited)

/Users/balkanskihaker/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/balkanskihaker/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/balkanskihaker/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/balkanskihaker/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed

In [18]:
submission = pd.DataFrame()
submission['Id'] = test_data['Id']
submission['SalePrice'] = prediction

In [19]:
submission.to_csv('submission.csv', index=False)